# Benchmarking on STM32-Cube.ai



In [479]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd
import numpy as np

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.wandb import wandb_model_DB, get_model_DB_run_id_from_architecture, get_architecture_from_model_DB_run_id

import wandb
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [480]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [481]:
models_dir = initialize()

In [482]:
automated = False

global model_name

#model_name = "mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV3"
model_name = "mobilenext_0.25_96_c3_o2_i1"



models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

c:\tiny_mlc\tiny_cnn\models


In [483]:
models_stm32_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_stm32_benchmark")
models_stm32_benchmark_path

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenext_0.25_96_c3_o2_i1/mobilenext_0.25_96_c3_o2_i1_stm32_benchmark')

In [484]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32tflm $models_tflite_path

In [485]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32tfl.exe

In [486]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32ai 

In [487]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32ai validate --name network -m $models_tflite_path --type tflite --compression none --verbosity 1 --workspace C:\Users\Susanne\AppData\Local\Temp\mxAI_workspace1067734059012002168110269332871471 --output $models_stm32_benchmark_path --allocate-inputs --allocate-outputs --mode stm32 --desc 115200 --with-report

# Helper functions

In [488]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [489]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [490]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [491]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

In [492]:
models_stm32_benchmark_file = models_dir.joinpath(model_name, "network_output", "network_validate_report.txt")
models_stm32_benchmark_file
models_stm32_benchmark_file_INT8 = models_dir.joinpath(model_name, "network_output", "network_validate_report_INT8.txt")
models_stm32_benchmark_file_INT8

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenext_0.25_96_c3_o2_i1/network_output/network_validate_report_INT8.txt')

In [493]:
def clean_benchmark_file(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                #line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [494]:
#! explorer $models_path

In [495]:
! code $models_stm32_benchmark_file

In [496]:
! code $models_stm32_benchmark_file_INT8

In [497]:
clean_lines = clean_benchmark_file(models_stm32_benchmark_file_INT8)
#clean_lines

In [498]:
def get_index_of_benchmark_lines(lines):
    model_lines_dict = {}
    for i, line in enumerate(lines):
        if "model name            :" in line:
            model_lines_dict["model_name"] = i
        elif "m_id   layer (type,original)" in line:
            model_lines_dict["m_id"] = i

        elif "model/c-model:" in line:
            model_lines_dict["model/c-model"]=i
            
        elif "activations size" in line:
            model_lines_dict["activations_size"] = i
        elif "weights size" in line:
            model_lines_dict["weights_size"] = i
        elif "macc                  :" in line:
            model_lines_dict["macc"] = i
        elif "duration        :" in line:
            model_lines_dict["duration"] = i
        elif "cycles/MACC     :" in line:
            model_lines_dict["cycles/MACC"] = i
        elif "counter         :" in line:
            model_lines_dict["counter"] = i

        elif "C-Arrays" in line:
            model_lines_dict["c-arrays"] = i
        elif "C-Layers" in line:
            model_lines_dict["c-layers"] = i
        elif "Inference time per node" in line:
            model_lines_dict["inference_times"] = i

        elif "Statistic per tensor" in line:
            model_lines_dict["statistics"] = i
        elif "Number of operations per c-layer" in line:
            model_lines_dict["number_of_operations"] = i
        elif "Number of operation types" in line:
            model_lines_dict["number_of_operation_types"] = i
        elif "Complexity report" in line:
            model_lines_dict["complexity_report"] = i
        elif "Evaluation report " in line:
            model_lines_dict["evaluation_report"] = i
    #print(model_lines_dict)
    return model_lines_dict


In [499]:
model_lines_dict = get_index_of_benchmark_lines(clean_lines)
model_lines_dict

{'m_id': 33,
 'model/c-model': 267,
 'model_name': 273,
 'activations_size': 277,
 'weights_size': 278,
 'macc': 279,
 'c-arrays': 283,
 'c-layers': 676,
 'duration': 1358,
 'cycles/MACC': 1360,
 'counter': 1361,
 'inference_times': 1366,
 'statistics': 1500,
 'number_of_operations': 1532,
 'number_of_operation_types': 1665,
 'complexity_report': 1675,
 'evaluation_report': 1772}

In [500]:
# model_name
name_line = clean_lines[model_lines_dict["model_name"]]
name = name_line.split(":")[1].split()[0].strip()
name

'mobilenext_0_25_96_c3_o2_i1_int8'

In [501]:
suffix=""
inference_stats = {}


In [502]:
if "int8" in name:
    suffix = "_INT8"

In [503]:
# weights_size
activations_line = clean_lines[model_lines_dict["activations_size"]]
activations = int(activations_line.split(":")[1].split()[0].strip())
inference_stats[f"stm32_activations_b{suffix}"]= activations
activations

224888

In [504]:
# weights_size
weights_line = clean_lines[model_lines_dict["weights_size"]]
weights = int(weights_line.split(":")[1].split()[0].strip())
inference_stats[f"stm32_weights_b{suffix}"]= weights
weights

159224

In [505]:
# MACC
macc_line = clean_lines[model_lines_dict["macc"]]
macc = int(macc_line.split(":")[1].strip())
inference_stats[f"stm32_macc_b{suffix}"]= macc
macc

17002419

In [506]:
# cycles/MACC

try:
    cycles_line = clean_lines[model_lines_dict["cycles/MACC"]]
    cycles_line = cycles_line.split(":")[1].strip()
    cycles = float(cycles_line)
    inference_stats[f"stm32_cycles_per_macc_b{suffix}"]= cycles
    cycles
except:
    print("cylces/MACC not reported")

In [507]:
# inference time
duration_line = clean_lines[model_lines_dict["duration"]]
duration_line = duration_line.split(":")[1].strip().split()[0].replace("ms", "")# .map(float)
duration = float(duration_line)
inference_stats[f"stm32_inference_ms{suffix}"]= duration
duration

274.197

In [508]:
inference_stats

{'stm32_activations_b_INT8': 224888,
 'stm32_weights_b_INT8': 159224,
 'stm32_macc_b_INT8': 17002419,
 'stm32_cycles_per_macc_b_INT8': 3.48,
 'stm32_inference_ms_INT8': 274.197}

In [509]:
m_id_lines = clean_lines[model_lines_dict["m_id"]: model_lines_dict["model/c-model"]]
m_id_lines

['m_id   layer (type,original)                        oshape                  param/size             macc               connected to   | c_size         c_macc              c_type                               \n',
 '------ -------------------------------------------- ----------------------- --------------- ----------- -------------------------- --- -------------- ------------------- ------------------------------------ \n',
 '0      serving_default_input_10 (Input, )           [b:1,h:96,w:96,c:3]                                                              | +40(+100.0%)   +248,835(+100.0%)   pad_oi8/conv2d_oi8[0, 1]             \n',
 '       conv2d_0 (Conv2D, DEPTHWISE_CONV_2D)         [b:1,h:96,w:96,c:3]     30/39               248,835   serving_default_input_10   | -39(-100.0%)   -248,835(-100.0%)   \n',
 '       nl_0_nl (Nonlinearity, DEPTHWISE_CONV_2D)    [b:1,h:96,w:96,c:3]                          27,648                   conv2d_0   |                -27,648(-100.0%)    \n',
 '--

In [510]:
# get inidices for splitting the dataframe
name_indices =[0]
name_indices.append(m_id_lines[0].find("layer (type,original)"))
name_indices.append(m_id_lines[0].find("oshape"))
name_indices.append(m_id_lines[0].find("param/size"))
name_indices.append(m_id_lines[0].find("c_size"))
name_indices.append(m_id_lines[0].find("c_macc"))
name_indices.append(m_id_lines[0].find("c_type"))
name_indices.append(None)
name_indices

[0, 7, 52, 76, 135, 150, 170, None]

In [511]:
# split_indices = [0]
# parts = m_id_lines[1].split(" ")
# print(parts)
# position = 0

# for part in parts:
#     position = position + len(part)
#     split_indices.append(position)
# split_indices.append(None)
# split_indices

In [512]:
#m_id_df.columns

In [513]:
# params_split_df = pd.DataFrame()
# params_split_df["text"] = m_id_df["param/size           macc               connected to   |"].str.strip()#.replace("|", "", inplace=True)#.split(" ",expand=True)
# params_split_df["text"] = params_split_df["text"].str.replace("\|", "")
# params_split_df["connected_to"] = params_split_df["text"].str.split().str[-1]
# params_split_df["macc"] = params_split_df["text"].str.split().str[-2]
# params_split_df["params_size"] = params_split_df["text"].str.split().str[-3]
# params_split_df["params"] = params_split_df["params_size"].str.split("/").str[0]
# params_split_df["size"] = params_split_df["params_size"].str.split("/").str[1]
# params_split_df.head()

In [517]:
suffix

# m_indices = [0, 7, 64, 87, 105, 114, 157, 160, 181, 200, None]
# m_indices_INT8 = [0, 6, 68, 113, 129, 139,  211, 227, 247, None]

# # if suffix =="_INT8":
# #     m_indices = m_indices_INT8

m_indices = name_indices

split_list =[]
for line in m_id_lines[0 :-3]:
    if "---" in line:
        pass
    else:
        split_list.append([line[m_indices[i]:m_indices[i+1]].strip() for i in range(len(m_indices)-1)])
#split_list

m_id_df = pd.DataFrame(split_list)
col_names = m_id_df.iloc[0]
m_id_df = m_id_df[1:]
m_id_df.columns = col_names

m_id_df["m_id"] = m_id_df["m_id"].str.strip()
m_id_df["m_id"].loc[m_id_df["m_id"]==""] = np.NAN
m_id_df["m_id"] = m_id_df["m_id"].fillna(method="ffill")
m_id_df.head()


,m_id,"layer (type,original)",oshape,param/size macc connected to |,c_size,c_macc,c_type
1,0,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",|,+40(+100.0%),"+248,835(+100.0%)","pad_oi8/conv2d_oi8[0, 1]"
2,0,"conv2d_0 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:96,w:96,c:3]","30/39 248,835 serving_default_input_10 |",-39(-100.0%),"-248,835(-100.0%)",
3,0,"nl_0_nl (Nonlinearity, DEPTHWISE_CONV_2D)","[b:1,h:96,w:96,c:3]","27,648 conv2d_0 |",,"-27,648(-100.0%)",
4,1,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:96,w:96,c:8]","32/56 221,192 nl_0_nl |",,,conv2d_oi8[2]
5,2,"conv2d_2 (Conv2D, CONV_2D)","[b:1,h:96,w:96,c:24]","216/288 1,769,496 conv2d_1 |",,,conv2d_oi8[3]


In [518]:
m_id_df.columns

Index(['m_id', 'layer (type,original)', 'oshape',
       'param/size             macc               connected to   |', 'c_size',
       'c_macc', 'c_type'],
      dtype='object', name=0)

In [ ]:


# split middle part
params_split_df = pd.DataFrame()
params_split_df

In [ ]:
params_split_df["text"] = m_id_df["param/size           macc               connected to   |"].str.strip()#.replace("|", "", inplace=True)#.split(" ",expand=True)
params_split_df["text"] = m_id_df["param/size             macc               connected to   |"].str.strip()

params_split_df["text"] = params_split_df["text"].str.replace("\|", "")
params_split_df["connected_to"] = params_split_df["text"].str.split().str[-1]
params_split_df["macc"] = params_split_df["text"].str.split().str[-2]
params_split_df["params_size"] = params_split_df["text"].str.split().str[-3]
params_split_df["params"] = params_split_df["params_size"].str.split("/").str[0]
params_split_df["size"] = params_split_df["params_size"].str.split("/").str[1]
params_split_df.head()


m_id_df["macc"] = params_split_df["macc"]
m_id_df["params"] = params_split_df["params"]
m_id_df["size"] = params_split_df["size"]
m_id_df["connected_to"] = params_split_df["connected_to"]
m_id_df.drop(["param/size           macc               connected to   |"], axis=1, inplace=True)

# clean up macc
m_id_df["macc"].loc[m_id_df["macc"]==""] = np.NAN
m_id_df["macc"] = m_id_df["macc"].str.replace(",", "")
m_id_df["macc"].fillna(0, inplace=True)
m_id_df["macc"] = m_id_df["macc"].astype(int)

m_id_df.drop(m_id_df[m_id_df["layer (type,original)"]==""].index, inplace=True)

# split layer name
layer_split_df = m_id_df["layer (type,original)"].str.split(" \(", expand=True)
layer_split_df[1] = layer_split_df[1].str.replace("\)", "")
#layer_split_df.head()

m_id_df["layer_name"] = layer_split_df[0]
m_id_df["type"] = layer_split_df[1]

m_id_df["params"] = m_id_df["params"].str.replace(",", "")
m_id_df["size"] = m_id_df["size"].str.replace(",", "")
m_id_df["params"].fillna(0, inplace=True)
m_id_df["size"].fillna(0, inplace=True)
m_id_df["params"] = m_id_df["params"].astype(int)
m_id_df["size"] = m_id_df["size"].astype(int)
m_id_df.reset_index(inplace=True, drop=False)

m_id_df.head()

In [ ]:
mask = m_id_df["m_id"].duplicated()
s1 = '.' + m_id_df[mask].groupby('m_id').cumcount().add(1).astype(str)
m_id_df.loc[mask, 'm_id'] += s1
#m_id_df["m_id"] = m_id_df["m_id"].astype(float)
m_id_df.head()


In [ ]:
#m_id_df.drop(m_id_df[m_id_df["layer (type,original)"]==""].index, inplace=True)

In [ ]:
m_id_df.info()

In [ ]:
#m_id_df.to_csv("m_id_df.csv")

In [ ]:
# m_id_df_slimm = m_id_df.copy()
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Input, "].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Placeholder, "].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Pad, PAD"].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type==""].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, HARD_SWISH"].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, DEPTHWISE_CONV_2D"].index, inplace=True)
# m_id_df_slimm.dropna(subset=["type"], inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, CONV_2D"].index, inplace=True)
# #m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type is None].index, inplace=True)
# m_id_df_slimm.reset_index(inplace=True)

# m_id_df_slimm.index +=1
# m_id_df_slimm

# Getting inference times per layer

In [ ]:
inference_lines = clean_lines[model_lines_dict["inference_times"]: model_lines_dict["statistics"]]
inference_lines

In [ ]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [ ]:
indices = [0, 8, 15,36,49, 60,79, None]
split_list =[]
for line in inference_lines[2 :-3]:
    if "---" in line:
        pass
    else:
        split_list.append([line[indices[i]:indices[i+1]].strip() for i in range(len(indices)-1)])

inference_df = pd.DataFrame(split_list)
col_names = inference_df.iloc[0]
inference_df = inference_df[1:]
inference_df.columns = col_names
inference_df["c_id"] = inference_df["c_id"].str.strip()
inference_df["dur (ms)"] = inference_df["dur (ms)"].astype(float)
inference_df["%"] = inference_df["%"].apply(string_percent_to_float)
inference_df["counters"] = inference_df["counters"].str.replace("[", "")
inference_df["counters"] = inference_df["counters"].str.replace("]", "")
inference_df["counters"] = inference_df["counters"].str.replace(",", "").astype(int)
inference_df.rename(columns={"name": "node_name", "type": "node_type"}, errors="raise", inplace=True)
#print(col_names)
inference_df

In [ ]:
inference_df.info()

In [ ]:
#inference_df.to_csv("inference_df.csv")

In [ ]:
merged_df = pd.merge(inference_df,m_id_df,on='m_id', how="outer")#inference_df.join(m_id_df_slimm, on="m_id", how="left")
merged_df#.head(20)

In [ ]:
# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, HARD_SWISH"].index, inplace=True)
# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, DEPTHWISE_CONV_2D"].index, inplace=True)

# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, CONV_2D"].index, inplace=True)
# merged_df.drop(merged_df[merged_df.type=="Eltwise, MINIMUM"].index, inplace=True)

# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, RELU"].index, inplace=True)
# #merged_df.drop(merged_df[merged_df["layer (type,original)"=="Nonlinearity, RELU"].index, inplace=True)
# merged_df.dropna(subset=["layer (type,original)"], inplace=True)
# merged_df.dropna(subset=["c_id"], inplace=True)
# merged_df["type"].fillna("", inplace=True)
# merged_df.drop_duplicates(subset=["layer (type,original)"], keep="first", inplace=True)
# #merged_df.drop("index", axis=1, inplace=True)
# merged_df.reset_index(inplace=True, drop=False)
# merged_df.rename(columns={"index": "_index"}, errors="raise", inplace=True)


In [ ]:
merged_df.columns

In [ ]:
merged_df.isnull().sum()

In [ ]:
merged_df["m_id"] =  merged_df["m_id"].astype(float)
merged_df.sort_values("m_id", inplace=True)
merged_df.reset_index(inplace=True, drop=True)
#merged_df.fillna(0, inplace=True)
merged_df.head()

In [ ]:
# combined_df = m_id_df_slimm.copy()
# combined_df["c_type"] = inference_df["type"]
# combined_df["dur (ms)"] = inference_df["dur (ms)"]
# combined_df["counters"] = inference_df["counters"]
# combined_df["node_name"] = inference_df["node_name"]
# combined_df

In [ ]:
#combined_df[["m_id", "layer_name", "type_x", "type_y", "dur (ms)"]]

In [ ]:
#inference_df["c_id"].unique()

In [ ]:
inference_stats

# Logging to wandb

In [ ]:
# Generate run ids
#id = wandb.wandb.sdk.lib.runid.generate_id()

id = get_model_DB_run_id_from_architecture(model_name)
PROJECT = "model_DB"

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        id = id, 
        resume="allow",
        )


# # Create a table
stm32_inference_df_table = wandb.Table(dataframe=inference_df)
stm32_int8_table = wandb.Table(dataframe= merged_df)

run.log({"stm32_inference_times_int8": stm32_inference_df_table})
run.log({"stm32_inference_int8": stm32_int8_table})

run.log(inference_stats)



wandb.finish()